In [1]:
#!pip install numpy

In [2]:
#!pip install flask

In [3]:
#!pip install mtcnn-0.1.0-py3-none-any.whl

In [4]:
#!pip install flask-mysqldb

In [5]:
#!pip install tqdm

In [2]:
import pandas as pd
import pickle
from flask import Flask, render_template, request, redirect, url_for, session, flash, jsonify
from werkzeug.utils import secure_filename
#from werkzeug.datastructures import FileStorage
from tensorflow.keras.models import load_model
from mtcnn import MTCNN
from flask_mysqldb import MySQL

Using TensorFlow backend.


In [3]:
#!pip install flask-mysqldb

In [4]:
from tqdm import tqdm_notebook
import cv2
import os
import numpy as np
from collections import Counter

In [14]:
app = Flask(__name__)
#model = pickle.load(open('model.pkl','rb'))
app.secret_key = 'xJ1\xd1\xc4\xcf\xed2C\xfc\x95\x92'

# db connect
app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = '1234'
app.config['MYSQL_DB'] = 'dbdeep'

mysql = MySQL(app)

@app.route("/")
def home():
    # db 연결
    cur = mysql.connection.cursor()
    cur.execute("select * from video_test")
    rows = cur.fetchall()
    cur.close()
    
    tag_list = []
    for obj in rows :
        tag = obj[2]
        tag_list.append(tag)
        
    tag_result = Counter(tag_list)
    result_list = tag_result.most_common(10)
    
    tag_lists = []
    for obj in result_list :
        tag = obj[0]        # tag_list
        tag_lists.append(tag)
        
    print(tag_lists)
    return render_template('main.html', tag_lists = tag_lists)

@app.route("/nav")
def nav(): 
    return render_template('nav.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":

        id = request.form['id']
        password = request.form['password']

        cur = mysql.connection.cursor()
        cur.execute("SELECT id, name, password, age, sex FROM user WHERE id = %s AND password = %s", (id, password))
        
        account = cur.fetchone()
        
        if account:
            session['login'] = True
            session['id'] = account[0]
            session['name'] = account[1]
            session['password'] = account[2]
            session['age'] = account[3]
            session['sex'] = account[4]

            return jsonify({"status": "success"}) # json형태로 상태를 리턴 해줌 
        else:
            return jsonify({"status": "fail"})
    return render_template('login.html')


@app.route('/logout')
def logout():
    session.pop('login', None)
    session.pop('id', None)
    session.pop('name', None)
    session.pop('password', None)
    session.pop('age', None)
    session.pop('sex', None)
    
    return render_template('main.html')

@app.route('/join', methods=['GET', 'POST'])
def join():
    if request.method == "POST":

        id = request.form['id']
        name = request.form['name']
        password = request.form['password']
        password_check = request.form['password_check']
        age = request.form['age']
        sex = request.form['sex']

        if(password == password_check):
            cur = mysql.connection.cursor()
            cur.execute("INSERT INTO user(id, name, password, age, sex) VALUES (%s, %s, %s, %s, %s)", (id, name, password, age, sex))
            mysql.connection.commit()
            cur.close()
            return jsonify({"status": "success"})
        else:
            return jsonify({"status": "fail"})

    return render_template('join.html')


@app.route('/mypage', methods=['GET', 'POST'])
def mypage():
    id = session['id']
    print("mypage id:",id)
    # db 연결
    cur = mysql.connection.cursor()
    cur.execute("select * from video_test where id = '%s'" % id)
    video = cur.fetchall()
    #print(video)
    cur.close()
    
    #이용내역
    video_list = []
    result_list = []
    for obj in video :
        video_name = obj[1] # 파일이름
        result = obj[3]     #결과
        
        video_list.append(video_name)
        result_list.append(result)
        
    #문의 내역
        cur = mysql.connection.cursor()
        cur.execute("select * from board where id = '%s'" % id)

        list = cur.fetchall()
    
    # edit_user    
    if request.method == "POST":
        name = request.form['name']
        password = request.form['password']
        password_check = request.form['password_check']
        

        if password == password_check:
            if name:
                name = name
                session['name'] = name
            else:
                name = session['name']

            if password:
                password = password
                session['password'] = password
            else:
                password = session['password']

            
            cur = mysql.connection.cursor()
            cur.execute("UPDATE user SET name=%s, password=%s WHERE id = %s", (name, password, id))
            mysql.connection.commit()
            cur.close()
            return jsonify({"status": "success"})
        
        else:
            return jsonify({"status": "fail"})

    return render_template('mypage.html', video_list = video_list, result_list = result_list, list = list)

@app.route("/manual")
def manual():                           
    return render_template('manual.html')


@app.route("/upload")
def upload():                           
    return render_template('upload.html')


@app.route("/fileUpload", methods = ['GET', 'POST'])
def upload_file():         
    if request.method == 'POST':
        
        f = request.files
        print("1",f)
        print("2",f['files'])
        print("3",f['files'].filename)
        f['files'].save("static/testMp4/" + secure_filename(f['files'].filename))
        
        # 2. 모델 불러오기
        model = load_model('keras_model.h5')
        #print(model)

        # 3. 모델 사용하기
        #yhat = model.predict_classes(xhat)
        def prediction_pipline(X, two_times=False):
            preds=[]
            #for model in tqdm_notebook(models):
            pred=model.predict(np.array(X))
            preds.append(pred)
            preds=sum(preds)/len(preds)
            if two_times:
                return larger_range(preds,2)
            else:
                return preds
        MAX_SKIP=10
        NUM_FRAME=150
        test_dir = 'static/testMp4/'
        filenames = f['files'].filename
        #prediction_filenames = filenames
        test_video_files = [test_dir + filenames]
        print('확인,,,',test_video_files)
        detector = MTCNN()
        def detect_face(img):
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            final = []
            detected_faces_raw = detector.detect_faces(img)
            if detected_faces_raw==[]:
                #print('no faces found')
                return []
            confidences=[]
            for n in detected_faces_raw:
                x,y,w,h=n['box']
                final.append([x,y,w,h])
                confidences.append(n['confidence'])
            if max(confidences)<0.9:
                return []
            max_conf_coord=final[confidences.index(max(confidences))]
            #return final
            return max_conf_coord
        def crop(img,x,y,w,h):
            x-=40
            y-=40
            w+=80
            h+=80
            if x<0:
                x=0
            if y<=0:
                y=0
            return cv2.cvtColor(cv2.resize(img[y:y+h,x:x+w],(150,150)),cv2.COLOR_BGR2RGB)
        def detect_video(video):
            v_cap = cv2.VideoCapture(video)
            v_cap.set(1, NUM_FRAME)
            success, vframe = v_cap.read()
            vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
            bounding_box=detect_face(vframe)
            if bounding_box==[]:
                count=0
                current=NUM_FRAME
                while bounding_box==[] and count<MAX_SKIP:
                    current+=1
                    v_cap.set(1,current)
                    success, vframe = v_cap.read()
                    vframe = cv2.cvtColor(vframe, cv2.COLOR_BGR2RGB)
                    bounding_box=detect_face(vframe)
                    count+=1
                if bounding_box==[]:
                    print('no faces found')
                    prediction_filenames.remove(video.replace('static/testMp4/',''))
                    return None
            x,y,w,h=bounding_box
            v_cap.release()
            return crop(vframe,x,y,w,h)
        test_X = []
        for video in tqdm_notebook(test_video_files):
            x=detect_video(video)
            if x is None:
                continue
            test_X.append(x)
        preds=prediction_pipline(np.array(test_X, dtype = np.float32),two_times=False).clip(0.35,0.65) 
        print("preds:",preds)
        print("preds2:",preds[0])
        print("preds3:",preds[0][0])
        preds = preds[0][0]
        
        # 진짜/가짜 t_f
        if(preds >=0.55 and preds <=1):
            t_f = '가짜영상'
        else :
            t_f = '진짜영상'
        
        # 진짜/가짜 percent
        if(preds<=0.09):
            percent = "99%"
        elif(preds<=0.19):
            percent = "90%"
        elif(preds <= 0.29):
            percent = "80%"
        elif(preds <= 0.39):
            percent = "70%"
        elif(preds <= 0.49):
            percent = "60%"
        elif(preds <= 0.59):
            percent = "50%"
        elif(preds <= 0.69):
            percent = "60%"
        elif(preds <= 0.79):
            percent = "70%"
        elif(preds <= 0.89):
            percent = "80%"
        elif(preds <= 0.95):
            percent = "90%"
        elif(preds <= 1):
            percent = "99%"

        print(percent)
        
        #db 연결
        # tag, source
        if session['login']:
            id = session['id']
        tag = request.form['tag']
        source = request.form['source']
        #print(tag)
        #print(source)
        
        # db 연결
        cur = mysql.connection.cursor()
        cur.execute("INSERT INTO video_test VALUES (%s, %s, %s, %s, now(), %s)", (id, filenames, tag, t_f, source))
        mysql.connection.commit()
        cur.close()


    return render_template('result.html', result = filenames, predict = percent, t_f = t_f)

@app.route("/result")
def result(result):
    print("result",result)
    return render_template('result.html', result = result, predict = preds, f = f)

@app.route("/stat")
def stat():
    # db 연결
    cur = mysql.connection.cursor()
    cur.execute("select * from video_test")
    rows = cur.fetchall()
    
   # rows data_list에 넣기
    data_list = []
    source_list = []
    tag_list = []

    for obj in rows :
        data = obj[3]         # 가짜/진짜
        source = obj[5]       # 출처
        tag = obj[2]
        
        data_list.append(data)
        source_list.append(source)
        tag_list.append(tag)
        
     # 원형 차트(가짜/진짜)    
    #print(data_list)   #label
    #print(len(data_list))  #count
    fake = data_list.count('가짜영상')
    real = data_list.count('진짜영상')
    count = [fake,real]
    
    # 출처 차트
    youtube = source_list.count('유튜브')
    naver = source_list.count('네이버')
    google = source_list.count('구글')
    daum = source_list.count('다음')
    etc = source_list.count('기타')
    source_count = [youtube, naver, google, daum, etc]
    
    # 태그 차트
    #print(tag_list)
    tag_result = Counter(tag_list)
    result_list = tag_result.most_common(10)
    tag_lists = []
    tag_count_lists = []
    for obj in result_list :
        tag = obj[0]        # tag_list
        tag_count = obj[1]  # tag_count
        tag_lists.append(tag)
        tag_count_lists.append(tag_count)
    
    # 나이대별 차트
    cur.execute("select * from user")
    user = cur.fetchall()
    age_list = []

    for obj in user :
        age = obj[3]         # 나이
        age_list.append(age)
    
    age_10 = age_list.count(10)
    age_20 = age_list.count(20)
    age_30 = age_list.count(30)
    age_40 = age_list.count(40)
    age_50 = age_list.count(50)
    age_60 = age_list.count(60)
    age_count = [age_10, age_20, age_30, age_40, age_50, age_60]
    
    
    return render_template('stat.html', result = data_list, count = count, source_count = source_count, tag_lists = tag_lists,
                          tag_count_lists = tag_count_lists, age_count = age_count)

@app.route("/intro")
def intro():
    return render_template('intro.html')

@app.route('/write', methods=['GET', 'POST'])
def write():
    if request.method == "POST":

        title = request.form['title']
        id = session['id']
        content = request.form['content']
        
        try:
            cur = mysql.connection.cursor()
            cur.execute("INSERT INTO board(title, id, content) VALUES (%s, %s, %s)", (title, id, content))
            mysql.connection.commit()
            cur.close()
            return jsonify({"status": "success"})
        except:
            return jsonify({"status": "fail"})

    return render_template('board_write.html')


@app.route('/list', methods=['GET'])
def board_list():
        cur = mysql.connection.cursor()
        cur.execute("SELECT * FROM board")

        list = cur.fetchall()
        
        if(len(list)/5) < 1:
            page = 1;
        else:
            if len(list)%5 == 0:
                page = len(list)/5
            else:
                page = (len(list)/5)+1

        return render_template("board.html",list = list, page = int(page))

if __name__ == "__main__":
    app.run(host="localhost", port="9003", threaded=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9003/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Mar/2020 14:58:35] "GET / HTTP/1.1" 200 -


['정치인', '힐러리', '연예인', '오바마', '대통령', '진쌤', '현빈', '여자배우', '선생님', '설현']


127.0.0.1 - - [05/Mar/2020 14:58:36] "GET / HTTP/1.1" 200 -


['정치인', '힐러리', '연예인', '오바마', '대통령', '진쌤', '현빈', '여자배우', '선생님', '설현']


...

127.0.0.1 - - [05/Mar/2020 15:01:30] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2020 15:01:33] "GET /mypage HTTP/1.1" 200 -


mypage id: aa


127.0.0.1 - - [05/Mar/2020 15:01:33] "GET /static/css/images/video.JPG HTTP/1.1" 404 -
127.0.0.1 - - [05/Mar/2020 15:01:34] "GET /static/testMp4/hil_fake.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/Mar/2020 15:01:34] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2020 15:01:34] "GET /static/testMp4/obama2.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/Mar/2020 15:01:34] "GET /static/testMp4/hil_fake.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/Mar/2020 15:01:34] "GET /static/testMp4/hil_fake.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/Mar/2020 15:01:35] "GET /static/testMp4/obama2.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/Mar/2020 15:01:35] "GET /static/testMp4/hil_fake.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/Mar/2020 15:01:35] "GET /static/testMp4/obama2.mp4 HTTP/1.1" 206 -
127.0.0.1 - - [05/Mar/2020 15:01:45] "GET /list HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2020 15:01:46] "GET /nav HTTP/1.1" 200 -
127.0.0.1 - - [05/Mar/2020 15:01:52] "GET / HTTP/1.1" 200 -


['정치인', '힐러리', '연예인', '오바마', '대통령', '진쌤', '현빈', '여자배우', '선생님', '설현']


127.0.0.1 - - [05/Mar/2020 15:01:53] "GET /nav HTTP/1.1" 200 -


In [109]:
age = [10, 20, 10, 30, 40, 60, 50]

In [30]:
a = 0.51
if(a>=0.55 and a <=1):
    print('가짜')

In [49]:
preds = 0.8

In [50]:
if(preds<=0.09):
    percent = "99%"
elif(preds<=0.19):
    percent = "90%"
elif(preds <= 0.29):
    percent = "80%"
elif(preds <= 0.39):
    percent = "70%"
elif(preds <= 0.49):
    percent = "60%"
elif(preds <= 0.59):
    percent = "50%"
elif(preds <= 0.69):
    percent = "60%"
elif(preds <= 0.79):
    percent = "70%"
elif(preds <= 0.89):
    percent = "80%"
elif(preds <= 0.95):
    percent = "90%"
elif(preds <= 1):
    percent = "99%"
    
print(percent)

80%


In [73]:
a = ['가짜','진짜', '가짜']

In [68]:
fake = a.count('가짜')
real = a.count('진짜')

In [74]:
a.count(a[0])

2

In [19]:
a = ['유튜브']

In [24]:
list = ['연예인', '연예인', '연예인', '정치인', '설현', '오바마', '오바마', '현빈', '남자배우', '여자배우', '영화배우', '힐러리', '선생님', '대통령', '여자배우', '현빈배우', '현빈']

In [45]:
from collections import Counter

In [66]:
result = Counter(list)

In [67]:
print(result)

Counter({'연예인': 3, '오바마': 2, '현빈': 2, '여자배우': 2, '정치인': 1, '설현': 1, '남자배우': 1, '영화배우': 1, '힐러리': 1, '선생님': 1, '대통령': 1, '현빈배우': 1})


In [51]:
Counter(list).values()

dict_values([3, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1])

In [69]:
result2 = Counter(list).values()
print(result2)

dict_values([3, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1])


In [64]:
item = list(result)

TypeError: 'list' object is not callable

In [80]:
result.elements()

In [87]:
result_list = result.most_common(10)

In [90]:
result_list

[('연예인', 3),
 ('오바마', 2),
 ('현빈', 2),
 ('여자배우', 2),
 ('정치인', 1),
 ('설현', 1),
 ('남자배우', 1),
 ('영화배우', 1),
 ('힐러리', 1),
 ('선생님', 1)]

In [92]:
result_list[1]

('오바마', 2)

In [97]:
tag_list = []
tag_count_list = []
for obj in result_list :
    tag = obj[0]        # tag_list
    tag_count = obj[1]  # tag_count
    tag_list.append(tag)
    tag_count_list.append(tag_count)
    print(tag)
print(tag_list)
print(tag_count_list)

연예인
오바마
현빈
여자배우
정치인
설현
남자배우
영화배우
힐러리
선생님
['연예인', '오바마', '현빈', '여자배우', '정치인', '설현', '남자배우', '영화배우', '힐러리', '선생님']
[3, 2, 2, 2, 1, 1, 1, 1, 1, 1]


In [133]:
from flask import Flask, request, session, redirect, url_for
app = Flask(__name__)
app.secret_key = 'any random string'

@app.route('/')
def index():
    if 'username' in session:
        username = session['username']
        return 'Logged in as ' + username + '<br>' + \
    "<b><a href = '/logout'>click here to log out</a></b>"
    
    return "You are not logged in <br><a href = '/login'></b>" + \
    "click here to log in</b></a>"

@app.route('/login', methods = ['GET', 'POST'])
def login():
    if request.method == 'POST':
        session['username'] = request.form['username']
        return redirect(url_for('index'))
    return '''
    
    <form action = "" method = "post">
       <p><input type = "text" name = "username"/></p>
       <p><input type = "submit" value = "Login"/></p>
    </form>
    '''
@app.route('/logout')
def logout():
   # remove the username from the session if it is there
    session.pop('username', None)
    return redirect(url_for('index'))


if __name__ == "__main__":
    app.run(host="localhost", port="9003")

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:9003/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Feb/2020 20:16:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 20:16:18] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 20:16:19] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [28/Feb/2020 20:16:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 20:16:21] "GET /logout HTTP/1.1" 302 -
127.0.0.1 - - [28/Feb/2020 20:16:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 20:16:23] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [28/Feb/2020 20:16:26] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [28/Feb/2020 20:16:26] "GET / HTTP/1.1" 200 -


In [ ]:
from flask import Flask, render_template, request, session, redirect
from flask_mysqldb import MySQL
app = Flask(__name__)

app.secret_key = 'xJ1\xd1\xc4\xcf\xed2C\xfc\x95\x92'

app.config['MYSQL_HOST'] = 'localhost'
app.config['MYSQL_USER'] = 'root'
app.config['MYSQL_PASSWORD'] = '12345'
app.config['MYSQL_DB'] = 'dbdeep'

mysql = MySQL(app)

@app.route('/')
def home():
    return render_template('main.html')

@app.route('/join', methods=['GET', 'POST'])
def join():
    if request.method == "POST":

        id = request.form['id']
        name = request.form['name']
        password = request.form['password']
        password_check = request.form['password_check']
        age = request.form['age']
        sex = request.form['sex']

        if(password == password_check):
            cur = mysql.connection.cursor()
            cur.execute("INSERT INTO user(id, name, password, age, sex) VALUES (%s, %s, %s, %s, %s)", (id, name, password, age, sex))
            mysql.connection.commit()
            cur.close()
            return 'success'
        else:
            return '비밀번호가 다릅니다.'

    return render_template('join.html')


@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "POST":

        id = request.form['id']
        password = request.form['password']
        print(id, password)

        cur = mysql.connection.cursor()
        cur.execute("SELECT id, name, age, sex FROM user WHERE id = %s AND password = %s", (id, password))
        
        account = cur.fetchone()
        
        if account:
            session['login'] = True
            session['id'] = account[0]
            session['name'] = account[1]
            session['age'] = account[2]
            session['sex'] = account[3]

            return redirect("/")
        else:
            return '로그인 실패'
    return render_template('login.html')


@app.route('/logout')
def logout():
    session.pop('login', None)
    session.pop('id', None)
    session.pop('name', None)
    session.pop('age', None)
    session.pop('sex', None)
    
    return render_template('main.html')

if __name__ == '__main__':
    app.run()